In [ ]:
yellow_path = "//wsl$/Ubuntu-20.04/home/afer/yellow/"
import sys; sys.path.append(yellow_path); sys.path.append(yellow_path + 'buteo/'); sys.path.append(yellow_path + 'buteo/machine_learning/');
import ml_utils

In [ ]:
import pandas as pd
import numpy as np
import os
import math

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

# Tensorflow
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPool2D, Conv2DTranspose, Concatenate, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import tensorflow_addons as tfa


os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
X1 = np.load('C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/antialiased_data/patches_singlepart/dtm_aeroe_singlepart.npy')
y = np.load('C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/antialiased_data/patches_singlepart/walls_singlepart_singlepart.npy')

In [ ]:
#X2 = np.load('C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/antialiased_data/patches_singlepart/dsm_aeroe_singlepart.npy')
X3 = np.load('C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/antialiased_data/patches_singlepart/hot_aeroe_singlepart.npy')

In [ ]:
X = np.stack((X1, X3), axis=3)
print(X.shape)

In [ ]:
#Remove the extra axis on the stacked array
X = X[..., 0]
print(X.shape)
print(y.shape)

In [ ]:
#If the data used is still with duplicates (previous version of extract_patches version)
meta_dups = pd.read_csv('C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/antialiased_data/patches_singlepart/dups_mask.csv')

dups_mask = ~meta_dups['og_fid'].duplicated()

X = X[dups_mask]
y = y[dups_mask]

print(X.shape)
print(y.shape)

#### **Adding ABSENCE data**

In [ ]:
#CREATE PURE PRESENCE BY REMOVING ABSENCES THAT MIGHT EXIST AROUND PATCH EXTRACTION BY THE WALLS

# presence_mask = np.sum(y, axis=(1, 2)) > 0
# X = X[presence_mask]
# y = y[presence_mask]

In [ ]:
# X_absence = np.load('/content/drive/MyDrive/Colab Notebooks/training_data/absence_only/dtm_aeroe_absence.npy')
# y_absence = np.load('/content/drive/MyDrive/Colab Notebooks/training_data/absence_only/absence_rasterized_absence.npy')

# len(X_absence)

In [ ]:
#CHANGE THIS NUMBER HERE TO CHANGE HOW MUCH ABSENCE DATA TO ADD

# #EXAMPLES:
# ADD 500 ABSENCES
# X_absence = X_absence[:500]
# y_absence = y_absence[:500]
# ADD 500 DIFFERENT ABSENCES
# X_absence = X_absence[500:1000]
# y_absence = y_absence[500:1000]

#default is to add no absence (if you check is empty)
# X_absence = X_absence[:0]
# y_absence = y_absence[:0]

In [ ]:
# add the presence and absence data together - if there is no absence then this does nothing, but is fine to run anyway
# X = np.concatenate([X, X_absence])
# y = np.concatenate([y, y_absence])



#### **Proceed here for PRESENCE only**

In [ ]:
# X = ml_utils.add_rotations(X)
# y = ml_utils.add_rotations(y)

In [ ]:
# multiply labels by 1000
y = np.multiply(y, 1000)


In [ ]:
meta = pd.read_csv('C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/antialiased_data/patches_dissolved/test_area/test_area_dissolved.csv')

train_mask = np.swapaxes(meta[meta["test_area"] == 0].values, 0, 1)[0]
test_mask = np.swapaxes(meta[meta["test_area"] == 1].values, 0, 1)[0]

In [ ]:
#add the masks to the X and y data in order to create the training and testing areas split
X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# print(len(X_train))
X_train = ml_utils.add_rotations(X_train)
X_test = ml_utils.add_rotations(X_test)
y_train = ml_utils.add_rotations(y_train)
y_test = ml_utils.add_rotations(y_test)
print(len(X_train))


In [ ]:
# shuffle the datasets. Could be interesting to see if this actually makes a difference, in which case dont run this cell
# np.random.seed(42)
# train_shuffle_mask = np.random.permutation(len(X_train))
# X_train = X_train[train_shuffle_mask]
# y_train = y_train[train_shuffle_mask]

# test_shuffle_mask = np.random.permutation(len(X_test))
# X_test = X_test[test_shuffle_mask]
# y_test = y_test[test_shuffle_mask]

In [ ]:
# check all the datasets match up in number
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

VERIFY THAT THESE TWO IMAGES CORRESPOND

In [ ]:
plt.imshow(np.reshape(X_train[2], (64,64)))

In [ ]:
plt.imshow(np.reshape(y_train[2], (64,64)))

# DEFINE AND RUN MODEL

In [ ]:
shape = (X_train.shape[1:])

In [ ]:
def define_model(shape, name, activation=tfa.activations.mish, sizes=[32, 64, 96, 128], double=True, pool="max", padding=["same", "valid"]):
    model_input = Input(shape=shape, name=name)
    model = Conv2D(sizes[0], kernel_size=3,
        padding=padding[0],
        activation=activation,
    )(model_input)
    if double:
        modelskip1 = Conv2D(sizes[0], kernel_size=3,
            padding=padding[0],
            activation=activation,
        )(model)
    if pool == 'max':
          model = MaxPool2D(padding=padding[0])(modelskip1)
    if pool == 'average':
          model = AveragePooling2D(padding=padding[0])(modelskip1)
    
    model = Conv2D(sizes[1], kernel_size=3,
        padding=padding[0],
        activation=activation,
    )(model)
    if double:
        modelskip2 = Conv2D(sizes[1], kernel_size=3,
            padding=padding[0],
            activation=activation,
        )(model)
    if pool == 'max':
        model = MaxPool2D(padding=padding[0])(modelskip2)
    if pool == 'average':
        model = AveragePooling2D(padding=padding[0])(modelskip2)
    
    model = Conv2D(sizes[2], kernel_size=3,
        padding=padding[0],
        activation=activation,
    )(model)
    if double:    
        model = Conv2D(sizes[2], kernel_size=3,
            padding=padding[0],
            activation=activation,
        )(model)

    model = Conv2DTranspose(sizes[1], kernel_size=3,
        strides=(2,2),
        padding=padding[0],
        activation=activation,
    )(model)

    model = Concatenate()([modelskip2, model])
    model = Conv2DTranspose(sizes[0], kernel_size=3,
        strides=(2,2),
        padding=padding[0],
        activation=activation,
    )(model)
    model = Concatenate()([modelskip1, model])


    output = Conv2D(1, kernel_size=3, padding=padding[0], activation='relu')(model)
    return Model(inputs=[model_input], outputs=output)

model = define_model(shape, name="vanilla")
model.summary()

In [ ]:
lr = 0.001

model.compile(
    optimizer=Adam(learning_rate=lr),
    loss='mse',
    metrics=['log_cosh', 'mae'],
)

def step_decay(epoch):
    initial_lrate = lr
    drop = 0.5
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

results = model.fit(
    X_train,
    y_train,
    epochs=10,
    validation_split=0.2,
    batch_size=32,
    callbacks=[
        LearningRateScheduler(step_decay),
        EarlyStopping(
            monitor="val_loss",
            patience=5,
            min_delta=10,
            restore_best_weights=True,
        ),
    ]
)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
mse_test = model.evaluate(X_test, y_test)

In [ ]:
import uuid
uid = uuid.uuid4().hex

out_directory = 'C:/Users/AFER/Documents/Projects/StoneWalls/Data/stonewalls/Aeroe/training_data/predictions/'
unique_filename = out_directory + uid + '.h5'
model.save(unique_filename)